In [2]:
import pandas as pd
import numpy as np

In [80]:
T_tweets = pd.read_csv('train.csv')

T_tweets['created_at'] = T_tweets['created_at'].str.replace('[','')
T_tweets['created_at'] = T_tweets['created_at'].str.replace(']','')
T_tweets['created_at'] = T_tweets['created_at'].str.replace('\'','')

T_tweets['date'], T_tweets['time'] = T_tweets['created_at'].str.split(',',1).str

T_tweets.drop(T_tweets.columns[1], axis=1, inplace=True)
T_tweets.head()


,id,text,intensity,date,time
0,834412638931083265,And also Karnataka bhavan jn towards Khodays ...,positive,2017-02-22,14:40:50
1,834412527366791169,Peak hour traffic at Mekhri circle underpass t...,caution,2017-02-22,14:40:23
2,834407314010435586,And also Anand Rao circle towards Keno theate...,positive,2017-02-22,14:19:40
3,834407256208728066,Peak hour traffic at Nayandahalli jn towards D...,caution,2017-02-22,14:19:27
4,834381488221220864,And also Krishna floor mil jn towards Okalipu...,positive,2017-02-22,12:37:03


In [82]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

T_tweets_test = pd.read_csv('tweets.csv')

y_pred = gnb.fit(T_tweets['text'], T_tweets['intensity']).predict(T_tweets_test['text'])

score = gnb.score((T_tweets['text'], T_tweets['intensity']),(T_tweets['intensity']))

ValueError: could not convert string to float: 'Slow moving traffic at Byappanahalli towards the city due to bus breakdown'